In [134]:
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow import keras
from tensorflow.keras.models import Sequential, load_model
import json

In [47]:
df = pd.read_csv("./csv/combined_mbti_df.csv")

df = df.drop_duplicates().dropna()

df.tail(5)

,mbti,function_pair,danceability_mean,danceability_stdev,energy_mean,energy_stdev,loudness_mean,loudness_stdev,mode_mean,mode_stdev,speechiness_mean,speechiness_stdev,acousticness_mean,acousticness_stdev,liveness_mean,liveness_stdev,valence_mean,valence_stdev,tempo_mean,tempo_stdev,instrumentalness_mean,instrumentalness_stdev,Cminor_count,CMajor_count,C#/Dbminor_count,C#/DbMajor_count,DMajor_count,D#_EbMajor_count,Eminor_count,EMajor_count,Fminor_count,FMajor_count,F#/Gbminor_count,GMajor_count,G#/Abminor_count,G#/AbMajor_count,Aminor_count,AMajor_count,A#/Bbminor_count,BMajor_count,Dminor_count,D#_Ebminor_count,Gminor_count,A#/BbMajor_count,F#/GbMajor_count,Bminor_count
4076,ESTJ,SJ,0.552889,0.155100,0.595611,0.137195,-7.224889,2.101033,0.500000,0.514496,0.076161,0.081395,0.299630,0.301594,0.130150,0.076705,0.364722,0.184167,129.453222,34.684798,0.041584,0.136504,1.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,0.0,2.0
4077,ESTJ,SJ,0.517780,0.142557,0.674940,0.182267,-7.902200,2.822676,0.600000,0.494872,0.052608,0.043208,0.146922,0.211668,0.212286,0.162003,0.360696,0.234499,114.934620,24.479429,0.197203,0.308495,1.0,3.0,2.0,0.0,4.0,0.0,1.0,4.0,3.0,4.0,1.0,2.0,1.0,2.0,3.0,3.0,0.0,5.0,1.0,1.0,2.0,2.0,1.0,4.0
4078,ESTJ,SJ,0.585313,0.181908,0.694375,0.173636,-5.307063,1.531874,0.562500,0.512348,0.090244,0.086054,0.153912,0.244173,0.157800,0.106532,0.522062,0.243778,122.609875,33.198061,0.000996,0.002927,0.0,1.0,0.0,2.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4079,ESTJ,SJ,0.636040,0.152382,0.652420,0.176042,-6.553020,2.813042,0.540000,0.503457,0.095376,0.088705,0.209508,0.222048,0.156414,0.123012,0.558718,0.277860,125.042880,29.284283,0.038939,0.149555,1.0,6.0,1.0,2.0,2.0,1.0,4.0,3.0,4.0,3.0,0.0,4.0,2.0,2.0,2.0,0.0,3.0,1.0,0.0,1.0,2.0,2.0,1.0,3.0
4080,ESTJ,SJ,0.640733,0.145205,0.678333,0.182482,-5.763733,2.075266,0.555556,0.502519,0.106978,0.077171,0.148807,0.165350,0.192504,0.126879,0.555711,0.231692,123.650378,27.039630,0.000131,0.000509,2.0,4.0,2.0,4.0,5.0,1.0,0.0,1.0,1.0,1.0,1.0,2.0,1.0,3.0,2.0,0.0,4.0,1.0,2.0,0.0,3.0,1.0,2.0,2.0


In [48]:
playlist_features = [
    "danceability_mean", "danceability_stdev",
    "energy_mean", "energy_stdev",
    "loudness_mean", "loudness_stdev",
    "mode_mean", "mode_stdev",
    "speechiness_mean", "speechiness_stdev",
    "acousticness_mean", "acousticness_stdev",
    "liveness_mean", "liveness_stdev",
    "valence_mean", "valence_stdev",
    "tempo_mean", "tempo_stdev",
    "instrumentalness_mean", "instrumentalness_stdev"
]

### CONVERT TO 4 LETTER COLUMNS

In [132]:
df["IE"] = df["mbti"].str[0].map({'I': 0, 'E': 1})
df["NS"] = df["mbti"].str[1].map({'N': 0, 'S': 1})
df["TF"] = df["mbti"].str[2].map({'T': 0, 'F': 1})
df["JP"] = df["mbti"].str[3].map({'J': 0, 'P': 1})

df.head(5)

,mbti,function_pair,danceability_mean,danceability_stdev,energy_mean,energy_stdev,loudness_mean,loudness_stdev,mode_mean,mode_stdev,speechiness_mean,speechiness_stdev,acousticness_mean,acousticness_stdev,liveness_mean,liveness_stdev,valence_mean,valence_stdev,tempo_mean,tempo_stdev,instrumentalness_mean,instrumentalness_stdev,Cminor_count,CMajor_count,C#/Dbminor_count,C#/DbMajor_count,DMajor_count,D#_EbMajor_count,Eminor_count,EMajor_count,Fminor_count,FMajor_count,F#/Gbminor_count,GMajor_count,G#/Abminor_count,G#/AbMajor_count,Aminor_count,AMajor_count,A#/Bbminor_count,BMajor_count,Dminor_count,D#_Ebminor_count,Gminor_count,A#/BbMajor_count,F#/GbMajor_count,Bminor_count,IE,NS,TF,JP
0,INFP,NF,0.557841,0.155011,0.553325,0.225178,-8.352591,3.273317,0.659091,0.479495,0.067341,0.061083,0.293469,0.286088,0.163402,0.086266,0.448884,0.221764,120.954795,32.165778,0.019971,0.081805,1.0,5.0,3.0,5.0,4.0,3.0,5.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,1,1
1,INFP,NF,0.587636,0.135644,0.556273,0.191642,-8.215697,3.356867,0.636364,0.488504,0.074273,0.083388,0.341434,0.267077,0.154542,0.112496,0.447603,0.216069,131.871242,33.216008,0.061041,0.177559,1.0,5.0,0.0,1.0,1.0,0.0,4.0,0.0,1.0,2.0,0.0,3.0,1.0,2.0,1.0,2.0,1.0,4.0,1.0,1.0,1.0,1.0,0.0,0.0,0,0,1,1
2,INFP,NF,0.677000,0.128009,0.851280,0.128336,-5.046100,2.180554,0.580000,0.498569,0.272546,0.176013,0.058085,0.074818,0.264520,0.208097,0.265030,0.170518,128.206020,26.927194,0.250358,0.331805,0.0,0.0,3.0,13.0,4.0,0.0,2.0,0.0,2.0,1.0,2.0,2.0,1.0,2.0,0.0,1.0,1.0,3.0,1.0,0.0,0.0,1.0,2.0,9.0,0,0,1,1
3,INFP,NF,0.517000,0.169477,0.513412,0.258345,-10.172833,4.935140,0.785714,0.415300,0.051510,0.033911,0.442163,0.367025,0.152645,0.095956,0.337845,0.211890,120.593571,34.335224,0.210927,0.294346,2.0,6.0,0.0,2.0,3.0,0.0,1.0,5.0,1.0,3.0,2.0,1.0,0.0,3.0,2.0,5.0,0.0,2.0,0.0,1.0,0.0,3.0,0.0,0.0,0,0,1,1
4,INFP,NF,0.560400,0.141450,0.445862,0.242592,-10.572240,5.685179,0.820000,0.388088,0.065178,0.058684,0.479677,0.361406,0.169092,0.126635,0.351366,0.195116,114.693080,30.034142,0.069537,0.206277,2.0,12.0,0.0,4.0,2.0,2.0,2.0,0.0,0.0,2.0,0.0,4.0,1.0,1.0,1.0,7.0,2.0,1.0,0.0,0.0,0.0,5.0,1.0,1.0,0,0,1,1


In [135]:
x = df[playlist_features]

scaler = StandardScaler()
x = scaler.fit_transform(x)

### TRAIN MODEL (BY EACH LETTER)

In [ ]:
def train_model(x, y):
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True)
    
    class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = dict(enumerate(class_weights))

    model = Sequential([
        keras.layers.Dense(64, activation='relu', input_shape=(x.shape[1],)),
        keras.layers.Dropout(0.2),

        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dropout(0.2),

        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )

    model.fit(x_train, y_train,
              epochs=30, 
              validation_split=0.1,
              class_weight=class_weights)
    
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    
    return model, accuracy, x_test, y_test


In [151]:
ie_model, ie_accuracy, x_ie_test, y_ie_test = train_model(x, df["IE"])
ns_model, ns_accuracy, x_ns_test, y_ns_test = train_model(x, df["NS"])
tf_model, tf_accuracy, x_tf_test, y_tf_test = train_model(x, df["TF"])
jp_model, jp_accuracy, x_jp_test, y_jp_test = train_model(x, df["JP"])

Epoch 1/30


C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.6147 - loss: 0.6587 - val_accuracy: 0.7094 - val_loss: 0.6275
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6904 - loss: 0.6092 - val_accuracy: 0.7094 - val_loss: 0.6080
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7117 - loss: 0.5768 - val_accuracy: 0.7250 - val_loss: 0.6034
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7067 - loss: 0.5911 - val_accuracy: 0.7125 - val_loss: 0.6040
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7123 - loss: 0.5636 - val_accuracy: 0.7156 - val_loss: 0.6017
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7205 - loss: 0.5654 - val_accuracy: 0.7094 - val_loss: 0.5970
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7191 - loss: 0.5554 - val_accuracy: 0.7250 - val_loss: 0.5930
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7304 - loss: 0.5602 - val_accuracy: 0.7156 - val_loss: 0.5901
Epo

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4724 - loss: 0.7941 - val_accuracy: 0.5250 - val_loss: 0.6938
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5611 - loss: 0.6920 - val_accuracy: 0.5500 - val_loss: 0.6735
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5732 - loss: 0.6774 - val_accuracy: 0.5813 - val_loss: 0.6720
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5607 - loss: 0.6772 - val_accuracy: 0.5906 - val_loss: 0.6585
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5857 - loss: 0.6706 - val_accuracy: 0.5781 - val_loss: 0.6646
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5826 - loss: 0.6711 - val_accuracy: 0.5844 - val_loss: 0.6610
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6026 - loss: 0.6599 - val_accuracy: 0.5906 - val_loss: 0.6606
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6272 - loss: 0.6521 - val_accuracy: 0.5938 - val_loss: 0.6617
Epo

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.5805 - loss: 0.6931 - val_accuracy: 0.7063 - val_loss: 0.6047
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7104 - loss: 0.5810 - val_accuracy: 0.6781 - val_loss: 0.5906
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7208 - loss: 0.5751 - val_accuracy: 0.6938 - val_loss: 0.5815
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7258 - loss: 0.5691 - val_accuracy: 0.7000 - val_loss: 0.5779
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7124 - loss: 0.5712 - val_accuracy: 0.6781 - val_loss: 0.5802
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7383 - loss: 0.5550 - val_accuracy: 0.7063 - val_loss: 0.5765
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7312 - loss: 0.5512 - val_accuracy: 0.7188 - val_loss: 0.5743
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7342 - loss: 0.5519 - val_accuracy: 0.7094 - val_loss: 0.5782
Epo

C:\Users\DELL\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


90/90 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5201 - loss: 0.7139 - val_accuracy: 0.6531 - val_loss: 0.6323
Epoch 2/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5670 - loss: 0.6824 - val_accuracy: 0.6656 - val_loss: 0.6286
Epoch 3/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6059 - loss: 0.6600 - val_accuracy: 0.6687 - val_loss: 0.6254
Epoch 4/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6053 - loss: 0.6599 - val_accuracy: 0.6531 - val_loss: 0.6221
Epoch 5/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5933 - loss: 0.6564 - val_accuracy: 0.6625 - val_loss: 0.6259
Epoch 6/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6270 - loss: 0.6449 - val_accuracy: 0.6594 - val_loss: 0.6257
Epoch 7/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5955 - loss: 0.6503 - val_accuracy: 0.6562 - val_loss: 0.6228
Epoch 8/30
90/90 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5969 - loss: 0.6603 - val_accuracy: 0.6625 - val_loss: 0.6182
Epo

In [153]:
model_info = {
    "ie_model": (ie_model, ie_accuracy),
    "ns_model": (ns_model, ns_accuracy),
    "tf_model": (tf_model, tf_accuracy),
    "jp_model": (jp_model, jp_accuracy),
}

### SAVE MODELS

In [154]:
# load record
with open("./models/accuracy_record.json", "r") as f:
    config = json.load(f)
    
def new_save(model, accuracy, name):
    print(accuracy, config[name])
    if accuracy > config[name]:
        config[name] = accuracy
        model.save(f"./models/{name}.keras")  

for name, (model, accuracy) in model_info.items():
    new_save(model, accuracy, name)

# save record
with open("./models/accuracy_record.json", "w") as f:
    json.dump(config, f, indent=4)

0.7359198927879333 0.75844806432724
0.5944930911064148 0.6382978723404256
0.7334167957305908 0.7484355568885803
0.5982478260993958 0.6070087609511889


### LOAD MODEL FROM SAVE

In [155]:
def load_from_save(name):
    return load_model(f"./models/{name}.keras")

# for name, (model, _) in model_info.items():
#     model = load_from_save(name)

ie_model = load_from_save("ie_model")
ns_model = load_from_save("ns_model")
tf_model = load_from_save("tf_model")
jp_model = load_from_save("jp_model")

### TEST PREDICTION

In [156]:
ie_pred = (ie_model.predict(x_ie_test) > 0.5).astype(int).reshape(-1)
ns_pred = (ns_model.predict(x_ns_test) > 0.5).astype(int).reshape(-1)
tf_pred = (tf_model.predict(x_tf_test) > 0.5).astype(int).reshape(-1)
jp_pred = (jp_model.predict(x_jp_test) > 0.5).astype(int).reshape(-1)

# Map to MBTI
def get_mbti(ie, ns, tf, jp):
    return f"{'E' if ie else 'I'}{'S' if ns else 'N'}{'F' if tf else 'T'}{'P' if jp else 'J'}"

pred_mbti = [get_mbti(ie_pred[i], ns_pred[i], tf_pred[i], jp_pred[i]) for i in range(len(ie_pred))]

actual_mbti = [
    get_mbti(y_ie_test.iloc[i], y_ns_test.iloc[i], y_tf_test.iloc[i], y_jp_test.iloc[i])
    for i in range(len(y_ie_test))
]

# So sánh
for p, a in zip(pred_mbti[:20], actual_mbti[:20]):
    print(f"Pred: {p:<10} | Actual: {a}")


25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Pred: ENTJ       | Actual: ENTJ
Pred: ISFJ       | Actual: ENFJ
Pred: ENTP       | Actual: ENTP
Pred: ISFJ       | Actual: INFJ
Pred: ESTP       | Actual: ESTJ
Pred: ESTP       | Actual: INTJ
Pred: ENTJ       | Actual: INTJ
Pred: ENTP       | Actual: ENFP
Pred: ISFJ       | Actual: INFP
Pred: INFJ       | Actual: ISFP
Pred: ESFP       | Actual: ENFJ
Pred: ESTP       | Actual: ESTP
Pred: ISFP       | Actual: INFJ
Pred: INTP       | Actual: INFP
Pred: ESFP       | Actual: ISFJ
Pred: INTP       | Actual: ISFP
Pred: ENTJ       | Actual: INTJ
Pred: ESFP       | Actual: ESFP
Pred: ESTP       | Actual: ESTP
Pred: ESTJ       | Actual: ISTP


### MORE TEST

In [146]:
mbti = "INFP"

test_df = pd.read_csv(f"./csv/{mbti}_df.csv")

test_data = test_df[playlist_features]

ie_pred = (ie_model.predict(test_data) > 0.5).astype(int).reshape(-1)
ns_pred = (ns_model.predict(test_data) > 0.5).astype(int).reshape(-1)
tf_pred = (tf_model.predict(test_data) > 0.5).astype(int).reshape(-1)
jp_pred = (jp_model.predict(test_data) > 0.5).astype(int).reshape(-1)

# Map to MBTI
def get_mbti(ie, ns, tf, jp):
    return f"{'E' if ie else 'I'}{'S' if ns else 'N'}{'F' if tf else 'T'}{'P' if jp else 'J'}"

res = []

for i in range(len(ie_pred)):
    mbti_pred = get_mbti(ie_pred[i], ns_pred[i], tf_pred[i], jp_pred[i])
    res.append(mbti_pred)

table = pd.DataFrame(res).drop_duplicates()
table


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


,0
0,ESTJ
